In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import nltk
from nltk.tokenize import RegexpTokenizer

In [2]:
# read raw data
raw_sample = pd.read_csv('./raw_sample.csv', engine='c')
ad_feature = pd.read_csv('./ad_feature.csv', engine='c')
user_profile = pd.read_csv('./user_profile.csv', engine = 'c')

In [3]:
# join samples
sample = raw_sample.join(ad_feature.set_index('adgroup_id'), on='adgroup_id')
sample = sample.join(user_profile.set_index('userid'), on='user')
sample = sample.rename(columns = {"adgroup_id": "item_id", "user": "user_id"})
sample = sample.drop('nonclk', axis=1)

In [4]:
# define unique function
def _unique(sample, fname):
    tmp_df = pd.DataFrame()
    tmp_df[fname] = sample[fname].unique()
    num = len(tmp_df)
    tmp_df['tmp_feature'] = range(num)
    sample = sample.join(tmp_df.set_index(fname), on=fname)
    sample.drop(fname, axis=1, inplace=True)
    sample = sample.rename(columns = {"tmp_feature": fname})
    return num, sample

# preprocess fequency
item2count = sample.groupby(['item_id']).size().reset_index(name='count').sort_values(by='count')
sample = sample.join(item2count.set_index('item_id'), on='item_id')

# preprocess and generate descripition
spr_features = ['user_id', 'item_id', 'pid', 'cate_id', 'campaign_id', 'customer', 'brand', 'cms_segid', \
       'cms_group_id', 'final_gender_code', 'age_level', 'pvalue_level', \
       'shopping_level', 'occupation', 'new_user_class_level ']
ctn_features = ['time_stamp', 'price', 'count']
label = ['clk']
description = []
for spr_f in spr_features:
    print(spr_f) 
    num, sample = _unique(sample, spr_f)
    num += 1
    sample[spr_f] = sample[spr_f].fillna(num - 1)
    sample[spr_f] = sample[spr_f].astype('int')
    description.append((spr_f, num, 'spr'))
for ctn_f in ctn_features:
    sample[ctn_f] = sample[ctn_f].fillna(0.0)
    min_v = np.min(sample[ctn_f])
    max_v = np.max(sample[ctn_f])
    sample[ctn_f] = sample[ctn_f].map(lambda x: (x - min_v)/(max_v - min_v))
    description.append((ctn_f, -1, 'ctn'))
for l in label:
    description.append((l, 2, 'label'))
description

user_id
item_id
pid
cate_id
campaign_id
customer
brand
cms_segid
cms_group_id
final_gender_code
age_level
pvalue_level
shopping_level
occupation
new_user_class_level 


[('user_id', 1141730, 'spr'),
 ('item_id', 846812, 'spr'),
 ('pid', 3, 'spr'),
 ('cate_id', 6770, 'spr'),
 ('campaign_id', 423437, 'spr'),
 ('customer', 255876, 'spr'),
 ('brand', 99816, 'spr'),
 ('cms_segid', 99, 'spr'),
 ('cms_group_id', 15, 'spr'),
 ('final_gender_code', 4, 'spr'),
 ('age_level', 9, 'spr'),
 ('pvalue_level', 5, 'spr'),
 ('shopping_level', 5, 'spr'),
 ('occupation', 4, 'spr'),
 ('new_user_class_level ', 6, 'spr'),
 ('time_stamp', -1, 'ctn'),
 ('price', -1, 'ctn'),
 ('count', -1, 'ctn'),
 ('clk', 2, 'label')]

In [6]:
x_pid0 = sample[sample['pid']==0][['item_id', 'user_id']]
x_pid1 = sample[sample['pid']==1][['item_id', 'user_id']] 

In [9]:
print(len(set(x_pid0['user_id']).intersection(set(x_pid1['user_id']))))

13767


In [5]:
# split dataset
N, K = 2000, 500
item2count = sample.groupby(['item_id']).size().reset_index(name='count').sort_values(by='count')
item_ids = list(item2count['item_id'])
counts = np.array(item2count['count'])

item_ids, counts = np.asarray(item_ids), np.asarray(counts)
hot_item_ids = item_ids[counts > N]
cold_item_ids = item_ids[np.logical_and(counts <= N, counts >= 3 * K)]
item_group = sample.groupby('item_id')
train_base = pd.DataFrame()
for item_id in hot_item_ids:
    df_hot = item_group.get_group(item_id).sort_values(by='time_stamp')
    train_base = train_base.append(df_hot, ignore_index=True)
train_warm_a, train_warm_b, train_warm_c, test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
for item_id in cold_item_ids:
    df_cold = item_group.get_group(item_id).sort_values(by='time_stamp')
    train_warm_a = train_warm_a.append(df_cold[: K], ignore_index=True)
    train_warm_b = train_warm_b.append(df_cold[K: 2*K], ignore_index=True)
    train_warm_c = train_warm_c.append(df_cold[2*K: 3*K], ignore_index=True)
    test = test.append(df_cold[3*K:], ignore_index=True)
save_dic = {
    'train_base': train_base.sort_values('time_stamp'),
    'train_warm_a': train_warm_a.sort_values('time_stamp'),
    'train_warm_b': train_warm_b.sort_values('time_stamp'),
    'train_warm_c': train_warm_c.sort_values('time_stamp'),
    'test': test.sort_values('time_stamp'),
    'description': description
}
for name, df in save_dic.items():
    print("{} size: {}".format(name, len(df)))
with open('./emb_warm_split_preprocess_taobao-ad.pkl', 'bw+') as f:
    pickle.dump(save_dic, f)

train_base size: 3592047
train_warm_a size: 270500
train_warm_b size: 270500
train_warm_c size: 270500
test size: 109712
description size: 19


In [ ]:
# Get the train sef for Meta-Embedding method
with open('./emb_warm_split_preprocess_taobao-ad.pkl', 'rb+') as f:
    data = pickle.load(f)
    df_base = data['train_base']
item2group = df_base.groupby('item_id')
train_a, train_b, train_c, train_d = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
for item_id, df_group in item2group:
    l, e = df_group.shape[0], df_group.shape[0] // 4     
    train_a = train_a.append(df_group.iloc[0: e,], ignore_index=True)
    train_b = train_b.append(df_group.iloc[e: 2 * e, ], ignore_index=True)
    train_c = train_c.append(df_group.iloc[2 * e: 3 * e, ], ignore_index=True)
    train_d = train_d.append(df_group.iloc[3 * e: 4 * e, ], ignore_index=True)
shuffle_idx = np.random.permutation(train_a.shape[0])
train_a = train_a.iloc[shuffle_idx]
train_b = train_b.iloc[shuffle_idx]
train_c = train_c.iloc[shuffle_idx]
train_d = train_d.iloc[shuffle_idx]
data["metaE_a"] = train_a
data["metaE_b"] = train_b
data["metaE_c"] = train_c
data["metaE_d"] = train_d
with open('./taobaoAD_data.pkl', 'wb+') as f:
    pickle.dump(data, f)